In [6]:
# for spotify 
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials, SpotifyOAuth

# os dependencies
import sys
import os

import pandas as pd 
import numpy as sns
import matplotlib.pyplot as plt 
import seaborn as sns
plt.rcParams['font.family'] = 'avenir'



### A Thought!
what if we use mongodb to store the post-analytics results of a particular user?

In [21]:
SPOTIPY_CLIENT_ID = "0be1da36f0594963b21a361ae5bd5d07"
SPOTIPY_CLIENT_SECRET = "093f6e330b414e4cbda05e95dbc0bef7"
SPOTIPY_REDIRECT_URI = 'http://localhost:8080/callback'

client_id = SPOTIPY_CLIENT_ID
client_secret = SPOTIPY_CLIENT_SECRET

In [23]:
import spotipy
from spotipy.oauth2 import SpotifyOAuth
import os 

# Set up the Spotipy OAuth object
sp_oauth = SpotifyOAuth(SPOTIPY_CLIENT_ID, SPOTIPY_CLIENT_SECRET, SPOTIPY_REDIRECT_URI, scope='playlist-read-private')

#Get user's authorization
token_info = sp_oauth.get_cached_token()
if not token_info:
    auth_url = sp_oauth.get_authorize_url()
    print(f'Please visit this URL to authorize the application: {auth_url}')
    response = input('Enter the URL you were redirected to: ')
    token_info = sp_oauth.get_access_token(response)

# Create a Spotipy object with the obtained token
sp = spotipy.Spotify(auth=token_info['access_token'])

# Get user's username
username = input('Enter Spotify username: ')

# Get user's playlists
playlists = sp.user_playlists(username)

# Display user's playlists
print(f"\n{username}'s playlists:")
for playlist in playlists['items']:
    print(f"  {playlist['name']} (id: {playlist['id']})")

# Get songs from a specific playlist
playlist_id = input('\nEnter the playlist ID to get songs from: ')
results = sp.playlist_tracks(playlist_id)

# Display songs from the playlist
print('\nSongs in the playlist:')
for track in results['items']:
    print(f"  {track['track']['name']} by {track['track']['artists'][0]['name']}")


Enter Spotify username: nocturnel99

nocturnel99's playlists:
  Your Top Songs 2023 (id: 37i9dQZF1Fa1IIVtEpGUcU)
  Ye (id: 2wLU1v4DfGM5iHn7D6FHmc)
  Neural Retrospection (id: 78Z28Kqp2mZvFDhGr4zZ2Q)
  Muscle Hypertrophy (id: 2HWHSWZvP7bf6jw41voDul)
  Shepard Tones (id: 4c5QsFfKkwGx3842nogeP6)
  Spirituality (id: 2eh9vqlJtrdBPAdKt2KzQV)
  RAP GOLDEN AGE (id: 6avcxlLLhOt8LoiuSgok2N)
  Brain fuck Code; (id: 6JsywVSM6Iav6lDHYYwwtL)
  Unsophisticated Code; (id: 6nlygRb8WVUhgbFBixQy2f)
  All Out 00's Telugu (id: 37i9dQZF1DXcrFZ8UTtxv9)
  Tranquility (id: 6IKepkvrPphQaDltrZjM6Y)
  Metal (id: 5YIg710T25jOQCoT8YxJXT)
  Psychedelics  (id: 3FF14kShJex0eMEBKJ0lm0)
  Complex Vocals (id: 12mN80CWfrsH8J2tHRQ2Q1)
  Nostalgia (id: 3Caue9V2okiV7aLpK4yNlt)
  Micheal Jackson (id: 3u8rPrpx0OnaH9Nk0PMo14)
  State of Mind (id: 7zOcTb2poIU4JgzJ4mJcSV)
  Hindi (id: 05U5u0gRLo3OLP1SQ70aeS)
  Telugu (id: 0ZPlDexP2aAjYHpm251MkL)
  Cyphers (id: 04LqQCvif58tI8SOWr1BVA)
  Stan (id: 08DNiQC9VmN6nrn5dJ7E1Q)
  Euphoria

In [19]:
for track in results['items']:
    print(f"  {track['track']['name']} by {track['track']['artists'][0]['name']}")

In [27]:
def format_song_list(results):
    song_list = []
    for track in results['items']:
        song_list.append(track['track']['name'])
    
    return song_list

In [38]:
SONGS = format_song_list(results)

In [45]:

def get_audio_features(song_name, client_id, client_secret):
    # Initialize Spotipy client credentials manager
    client_credentials_manager = SpotifyClientCredentials(client_id=client_id, client_secret=client_secret)
    sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

    # Search for the song by name
    results = sp.search(q=song_name, type='track', limit=1)

    # Check if any tracks were found
    if results['tracks']['items']:
        track_id = results['tracks']['items'][0]['id']
        
        # Get audio features for the track
        audio_features = sp.audio_features(track_id)
        
        if audio_features:
            # Extract the relevant audio features
            features = audio_features[0]
            feature_names = [
                'danceability', 'energy', 'key', 'loudness', 'mode', 
                'speechiness', 'acousticness', 'instrumentalness', 
                'liveness', 'valence', 'tempo', 'duration_ms'
            ]
            
            # Create a list of audio feature values
            feature_values = [features[feature] for feature in feature_names]
            
            return feature_values
        else:
            return None  # Audio features not found for the track
    else:
        return None  # Track not found

# Example usage
attributes = ['Danceability', 'Energy', 'Key', 'Loudness', 'Mode', 
             'Speechiness', 'Acousticness', 'Instrumentalness', 
             'Liveness', 'Valence', 'Tempo', 'Duration (ms)']
SONG_DICTIONARY = {}

for song in SONGS:
    audio_features_dictionary = {}
    print(f"Fetching {song}, ",end=" ")
    audio_features = get_audio_features(song_name, client_id, client_secret)
    if audio_features:
        for feature, value in zip(attributes, audio_features):
            audio_features_dictionary[feature] = value
        
        SONG_DICTIONARY[song] = audio_features_dictionary
        
    else:
        print(f"UnAvailable")
    
    print("Done!")
    

# print(SONG_DICTIONARY)


Fetching Gold Digger,  Done!
Fetching All Falls Down,  Done!
Fetching Hurricane,  Done!
Fetching Life Of The Party (with André 3000),  Done!
Fetching Jail,  Done!
Fetching Come to Life,  Done!
Fetching Believe What I Say,  Done!
Fetching No Child Left Behind,  Done!
Fetching God Breathed,  Done!
Fetching Lord I Need You,  Done!
Fetching Jail pt 2,  Done!
Fetching Good Morning,  Done!
Fetching Stronger,  Done!
Fetching Can't Tell Me Nothing,  Done!
Fetching Famous,  Done!
Fetching Hell Of A Life,  Done!
Fetching All Of The Lights,  Done!
Fetching I Wonder,  Done!
Fetching Wash Us In The Blood,  Done!
Fetching Eazy,  Done!
Fetching Violent Crimes,  Done!
Fetching Follow God,  Done!
Fetching Use This Gospel,  Done!
Fetching Jesus Is Lord,  Done!
Fetching Feedback,  Done!
Fetching Father Stretch My Hands Pt. 1,  Done!
Fetching Jesus Lord,  Done!
Fetching New Again,  Done!
Fetching Moon,  Done!
Fetching Off The Grid,  Done!
Fetching Otis,  Done!
Fetching Saint Pablo,  Done!
Fetching Champio

In [54]:
audio_features_df = pd.DataFrame(SONG_DICTIONARY).T.reset_index()
cols = audio_features_df.columns 
cols = list(cols)
cols[0] = 'song'
audio_features_df.columns = cols
audio_features_df

,song,Danceability,Energy,Key,Loudness,Mode,Speechiness,Acousticness,Instrumentalness,Liveness,Valence,Tempo,Duration (ms)
0,Gold Digger,0.825,0.652,1.0,-3.183,0.0,0.0802,0.581,0.0,0.0931,0.931,95.977,233713.0
1,All Falls Down,0.825,0.652,1.0,-3.183,0.0,0.0802,0.581,0.0,0.0931,0.931,95.977,233713.0
2,Hurricane,0.825,0.652,1.0,-3.183,0.0,0.0802,0.581,0.0,0.0931,0.931,95.977,233713.0
3,Life Of The Party (with André 3000),0.825,0.652,1.0,-3.183,0.0,0.0802,0.581,0.0,0.0931,0.931,95.977,233713.0
4,Jail,0.825,0.652,1.0,-3.183,0.0,0.0802,0.581,0.0,0.0931,0.931,95.977,233713.0
5,Come to Life,0.825,0.652,1.0,-3.183,0.0,0.0802,0.581,0.0,0.0931,0.931,95.977,233713.0
6,Believe What I Say,0.825,0.652,1.0,-3.183,0.0,0.0802,0.581,0.0,0.0931,0.931,95.977,233713.0
7,No Child Left Behind,0.825,0.652,1.0,-3.183,0.0,0.0802,0.581,0.0,0.0931,0.931,95.977,233713.0
8,God Breathed,0.825,0.652,1.0,-3.183,0.0,0.0802,0.581,0.0,0.0931,0.931,95.977,233713.0
9,Lord I Need You,0.825,0.652,1.0,-3.183,0.0,0.0802,0.581,0.0,0.0931,0.931,95.977,233713.0


### Analysis Pipeline